# QAOA for Food Production Optimization

## Learning Objectives
By the end of this notebook, you will:
1. Implement QAOA specifically for F×C food production problems
2. Code simplified versions of the QAOA methods used in OQI_Project
3. Understand multi-objective QUBO formulation for agriculture
4. Build the recursive QAOA techniques from the codebase
5. Compare quantum vs classical approaches on food allocation

## Real-World Context: OQI_Project QAOA Methods
This tutorial teaches the actual QAOA techniques used in the QOptimizer:
- **`optimize_with_recursive_qaoa_merge`**: Breaking large farm-food problems into manageable pieces
- **Multi-objective scoring**: Balancing nutrition, sustainability, affordability, environmental impact
- **QUBO conversion**: Transform food constraints into quantum-ready problems
- **Hybrid approaches**: Quantum optimization with classical fallback

## Prerequisites
Complete the Fundamentals for Food Production notebook first.

---

In [2]:
# Essential imports for food production QAOA
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import List, Tuple, Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass
import itertools
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Define the optimization objectives used in OQI_Project
class OptimizationObjective(Enum):
    """Types of optimization objectives for food production."""
    NUTRITIONAL_VALUE = "nutritional_value"
    NUTRIENT_DENSITY = "nutrient_density"
    ENVIRONMENTAL_IMPACT = "environmental_impact"
    AFFORDABILITY = "affordability"
    SUSTAINABILITY = "sustainability"

@dataclass
class FoodData:
    """Food characteristics for optimization."""
    name: str
    nutritional_value: float
    nutrient_density: float
    environmental_impact: float
    affordability: float
    sustainability: float
    
    def get_score(self, objective: OptimizationObjective) -> float:
        """Get score for specific objective."""
        return getattr(self, objective.value)

# Sample food data based on OQI_Project
SAMPLE_FOODS = {
    'Wheat': FoodData('Wheat', 0.7, 0.6, 0.3, 0.8, 0.7),
    'Rice': FoodData('Rice', 0.6, 0.5, 0.4, 0.9, 0.6),
    'Soybeans': FoodData('Soybeans', 0.9, 0.8, 0.2, 0.6, 0.8),
    'Corn': FoodData('Corn', 0.5, 0.4, 0.35, 0.85, 0.65),
    'Potatoes': FoodData('Potatoes', 0.4, 0.6, 0.25, 0.95, 0.75)
}

print("✓ Food production QAOA environment ready!")
print(f"Available foods: {list(SAMPLE_FOODS.keys())}")
print(f"Optimization objectives: {[obj.value for obj in OptimizationObjective]}")

✓ Food production QAOA environment ready!
Available foods: ['Wheat', 'Rice', 'Soybeans', 'Corn', 'Potatoes']
Optimization objectives: ['nutritional_value', 'nutrient_density', 'environmental_impact', 'affordability', 'sustainability']


## 1. Food Production QUBO Formulation

In the OQI_Project, we optimize F farms × C foods with multi-objective scoring. Let's implement the QUBO formulation used in the actual codebase.

### Problem Structure
- **Decision Variables**: Binary variables representing whether each farm grows each type of food
- **Objective**: Maximize weighted sum of scores across nutritional value, sustainability, affordability, and environmental impact
- **Constraints**: Farm land capacity limits, food diversity requirements, minimum production levels

### QUBO Matrix Construction
The QUBO matrix Q encodes our optimization problem where:
- **Diagonal terms**: Represent individual farm-food combination benefits and linear constraint penalties
- **Off-diagonal terms**: Capture interactions between different farm-food allocation decisions
- **Penalty terms**: Enforce constraint violations through quadratic penalty methods

Your task is to build a class that can convert our multi-objective food production problem into a format suitable for quantum optimization algorithms.

---

In [3]:
class FoodProductionQUBO:
    """QUBO formulation for food production optimization - simplified version of OQI_Project approach."""
    
    def __init__(self, farms: List[str], foods: Dict[str, FoodData]):
        """
        Initialize the QUBO formulation for food production optimization.
        
        In this constructor, you need to set up the foundational data structures that will represent
        our farm-food optimization problem. Think of this as creating the mathematical framework
        that will hold all the information about which farms can grow which foods and how valuable
        each combination is.
        
        Your task is to store the input parameters and calculate the dimensions of our optimization problem.
        Remember that we have F farms and C different food types, creating F×C possible farm-food 
        combinations. Each combination needs a binary decision variable (0 = don't grow, 1 = grow).
        
        You'll also need to initialize the QUBO matrix Q, which is the heart of our optimization.
        This matrix encodes both the benefits of growing foods and the penalties for violating constraints.
        Make sure to create it as a square matrix with the right dimensions to hold all variable interactions.
        """
        
        self.farms = farms
        self.foods = foods  
        self.F = len(farms)
        self.C = len(foods)

        self.num_variables = self.F * self.C
        # Initialize the QUBO matrix as an upper triangular matrix
        self.Q = np.zeros((self.num_variables, self.num_variables), dtype=float)
        # We only need the upper triangular part since Q is symmetric
        self.food_names = list(foods.keys())
        
    def get_variable_index(self, farm_idx: int, food_idx: int) -> int:
        """
        Convert two-dimensional farm-food indices into a single linear index.
        
        Since our QUBO matrix is one-dimensional but our problem is naturally two-dimensional
        (farms × foods), we need a systematic way to map between these representations.
        Think of this like converting row-column coordinates in a spreadsheet to a single
        cell number when reading left-to-right, top-to-bottom.
        
        Your mapping function should ensure that each farm-food combination gets a unique
        index number, and that you can reliably convert back and forth between the 2D and 1D
        representations.
        """
        return farm_idx * self.C + food_idx
    
    def get_farm_food_from_index(self, var_idx: int) -> Tuple[int, int]:
        """
        Convert a linear variable index back to farm and food indices.
        
        This is the reverse operation of get_variable_index. Given a single number representing
        a variable in our QUBO formulation, you need to figure out which farm and which food
        it corresponds to. This is essential for interpreting solutions and understanding which
        farm-food allocations are being recommended.
        
        Think about how you can reverse your indexing formula using mathematical operations
        like integer division and modulo arithmetic.
        """
        farm_idx = var_idx // self.C
        food_idx = var_idx % self.C
        return farm_idx, food_idx
    
    def set_multi_objective_weights(self, weights: Dict[str, float]):
        """
        Configure the objective function with weights for different optimization goals.
        
        This method implements the multi-objective scoring system used in the OQI_Project,
        where we balance multiple competing objectives like nutritional value, sustainability,
        affordability, and environmental impact. Each farm-food combination has scores for
        these different objectives, and you need to combine them into a single weighted score.
        
        Your task is to iterate through every possible farm-food combination, calculate its
        weighted benefit score, and store this in the diagonal of the QUBO matrix. Remember
        that environmental impact should be minimized (subtracted) while other objectives
        should be maximized (added). Also remember that QUBO formulation minimizes the objective,
        so you'll need to negate your benefit scores.
        """
        for farm_idx in range(self.F):
            for food_idx in range(self.C):
                var_idx = self.get_variable_index(farm_idx, food_idx)
                
                # Calculate weighted score for this farm-food combination
                food_name = self.food_names[food_idx]
                food_data = self.foods[food_name]
                
                weighted_score = 0.0
                for obj in OptimizationObjective:
                    obj_value = food_data.get_score(obj)
                    weight = weights.get(obj.value, 0.0)
                    
                    if obj == OptimizationObjective.ENVIRONMENTAL_IMPACT:
                        weighted_score -= weight * obj_value  # Minimize environmental impact
                    else:
                        weighted_score += weight * obj_value  # Maximize other objectives
                
                # Negative because QUBO minimizes but we want to maximize score
                self.Q[var_idx, var_idx] = -weighted_score
    
    
    def add_farm_capacity_constraints(self, farm_capacities: Dict[str, int], penalty_weight: float = 5.0):
        """
        Enforce farm land capacity limits using quadratic penalty methods.
        
        Real farms have limited land and can only grow a certain number of different food types.
        In QUBO formulation, we enforce constraints by adding penalty terms that make constraint
        violations expensive. When a farm tries to grow more foods than its capacity allows,
        the penalty terms increase the objective value, making such solutions less attractive.
        
        You need to implement the mathematical formula for quadratic penalties. For a constraint
        like "sum of foods ≤ capacity", the penalty takes the form of quadratic interactions
        between variables on the same farm. This creates both linear terms (diagonal elements)
        and quadratic terms (off-diagonal elements) in your QUBO matrix.
        
        Think about how to systematically add these penalty terms for each farm while ensuring
        you maintain the upper triangular structure of the QUBO matrix.

        Penalty: λ * (sum_f y_{fc} - capacity)^2
        """
        for farm_idx, farm in enumerate(self.farms):
            capacity = farm_capacities.get(farm, 1)
            
            # Quadratic penalty for exceeding capacity
            for food_idx1 in range(self.C):
                var_idx1 = self.get_variable_index(farm_idx, food_idx1)
                
                # Linear penalty terms
                self.Q[var_idx1, var_idx1] += penalty_weight * (1 - 2 * capacity)
                
                # Quadratic interaction terms
                for food_idx2 in range(food_idx1 + 1, self.C):
                    var_idx2 = self.get_variable_index(farm_idx, food_idx2)
                    # Upper triangular matrix
                    self.Q[var_idx1, var_idx2] += 2 * penalty_weight
    
    def add_diversity_constraints(self, min_farms_per_food: int = 1, penalty_weight: float = 3.0):
        """
        Ensure food security by requiring multiple farms to grow each food type.
        
        From a food security perspective, we don't want all of one food type grown on just
        one farm. If that farm has problems (weather, disease, etc.), we could lose that entire
        food supply. This constraint encourages spreading each food type across multiple farms.
        
        Similar to capacity constraints, you'll implement this using quadratic penalty methods.
        The penalty should activate when fewer than the minimum number of farms are growing
        a particular food type. This requires adding interactions between all farm variables
        for each food type.
        
        Consider how to structure the loops and penalty calculations to encourage diversity
        while maintaining the mathematical correctness of the QUBO formulation.
    
        Penalty: λ * max(0, min_farms_per_food - sum_f y_{fc})^2
        """
        for food_idx in range(self.C):
            # Penalty for not meeting minimum diversity
            for farm_idx1 in range(self.F):
                var_idx1 = self.get_variable_index(farm_idx1, food_idx)
                
                # Linear penalty terms  
                self.Q[var_idx1, var_idx1] += penalty_weight * (1 - 2 * min_farms_per_food)
                
                # Quadratic interaction terms
                for farm_idx2 in range(farm_idx1 + 1, self.F):
                    var_idx2 = self.get_variable_index(farm_idx2, food_idx)
                    # Upper triangular matrix
                    self.Q[var_idx1, var_idx2] += 2 * penalty_weight
    
    def evaluate_solution(self, solution: np.ndarray) -> float:
        """
        Calculate the total objective value for a given binary solution.
        
        Given a solution vector where each element is 0 or 1 (representing whether each
        farm-food combination is selected), you need to calculate what the total objective
        value would be. This involves computing the quadratic form that combines the benefits
        of selected combinations with any constraint violation penalties.
        
        Remember that your QUBO matrix Q is stored in upper triangular form for efficiency,
        so you'll need to create the full symmetric matrix before computing the quadratic form.
        The mathematical operation you're performing is solution^T × Q × solution.
        """
        full_Q = self.Q + self.Q.T - np.diag(np.diag(self.Q))
        return solution.T @ full_Q @ solution
    
    def get_solution_description(self, solution: np.ndarray) -> str:
        """
        Create a human-readable description of a solution.
        
        Raw binary vectors aren't very intuitive for understanding what a solution means
        in terms of actual farm-food allocations. This method should translate the mathematical
        solution back into meaningful statements like "Farm A grows Wheat and Corn" or
        "Farm B grows Soybeans".
        
        You'll need to examine each element of the solution vector, identify which farm-food
        combinations are selected (value = 1), and format this information in a clear,
        readable way that stakeholders can understand.
        """
        description = []
        for var_idx, value in enumerate(solution):
            if value == 1:
                farm_idx, food_idx = self.get_farm_food_from_index(var_idx)
                farm_name = self.farms[farm_idx]
                food_name = list(self.foods.keys())[food_idx]
                description.append(f"{farm_name} grows {food_name}")
        return ", ".join(description) if description else "No foods selected"

# Student Task: Create and test a small QUBO problem instance
print("Setting up Food Production QUBO Test")
print("===================================")

"""
Your task: Set up a test problem to verify your QUBO formulation works correctly.

1. Create a test problem using a subset of farms and foods:
   - Select the first 3 foods from your SAMPLE_FOODS dictionary
   - Create a list of 3 farm names (like "Farm_A", "Farm_B", "Farm_C")

2. Initialize your QUBO class with this test data and verify the setup:
   - Create a FoodProductionQUBO instance
   - Print the problem dimensions to confirm everything looks right

3. Configure a sustainability-focused optimization:
   - Create a weights dictionary that emphasizes sustainability and environmental factors
   - Maybe use weights like: sustainability (0.25), nutritional value (0.25), with smaller weights for other objectives
   - Apply these weights to your QUBO instance

4. Add realistic constraints:
   - Set up farm capacity constraints (maybe each farm can grow 2 different foods maximum)
   - Add diversity constraints (maybe each food should be grown on at least 1 farm)
   - Use reasonable penalty weights (try values around 3.0-5.0)

5. Print a summary showing:
   - QUBO matrix dimensions
   - The objective weights you're using
   - The constraint parameters you've set

This will give you a working QUBO formulation that you can use in the next sections!
"""

# Write your code here to set up the test problem:

test_farms = ['North_Farm', 'South_Farm', 'East_Farm']
test_foods = {name: data for name, data in list(SAMPLE_FOODS.items())[:3]}

qubo = FoodProductionQUBO(test_farms, test_foods)
print(f"Problem size: {qubo.F} farms × {qubo.C} foods = {qubo.num_variables} variables")

# Set multi-objective weights (sustainability focus)
sustainability_weights = {
    'nutritional_value': 0.25,
    'nutrient_density': 0.20,
    'environmental_impact': 0.15,
    'affordability': 0.15,
    'sustainability': 0.25
}

qubo.set_multi_objective_weights(sustainability_weights)

# Add constraints
farm_capacities = {'North_Farm': 2, 'South_Farm': 1, 'East_Farm': 2}
qubo.add_farm_capacity_constraints(farm_capacities, penalty_weight=4.0)
qubo.add_diversity_constraints(min_farms_per_food=1, penalty_weight=2.0)

print(f"QUBO matrix constructed: {qubo.Q.shape}")
print(f"Objective weights: {sustainability_weights}")
print(f"Farm capacities: {farm_capacities}")

Setting up Food Production QUBO Test
Problem size: 3 farms × 3 foods = 9 variables
QUBO matrix constructed: (9, 9)
Objective weights: {'nutritional_value': 0.25, 'nutrient_density': 0.2, 'environmental_impact': 0.15, 'affordability': 0.15, 'sustainability': 0.25}
Farm capacities: {'North_Farm': 2, 'South_Farm': 1, 'East_Farm': 2}


## 2. QAOA Implementation for Food Production

Now let's implement the QAOA algorithm specifically for our food production QUBO problem. This follows the structure used in the OQI_Project's QAOA solvers.

### QAOA Circuit for Food Production:
1. **Initial State**: Create uniform superposition over all possible farm-food combinations
2. **Problem Hamiltonian**: Apply phase rotations based on the QUBO objective function
3. **Mixer Hamiltonian**: Apply X rotations to explore different allocation possibilities  
4. **Parameter Optimization**: Use classical optimization to find the best rotation angles

### Key Concepts:
- **Quantum Superposition**: Allows exploring multiple farm-food allocations simultaneously
- **Phase Encoding**: QUBO objective values become quantum phases that guide the optimization
- **Variational Approach**: Classical optimizer tunes quantum circuit parameters for best results

---

In [8]:
class FoodProductionQAOA:
    """QAOA implementation for food production optimization - based on OQI_Project methods."""
    
    def __init__(self, qubo: FoodProductionQUBO, p: int = 1):
        """
        Initialize the QAOA algorithm for solving food production optimization problems.
        
        The QAOA (Quantum Approximate Optimization Algorithm) is a variational quantum algorithm
        that alternates between applying a problem Hamiltonian (encoding your objective function)
        and a mixer Hamiltonian (allowing exploration of different solutions). Think of it as
        a quantum version of simulated annealing that uses quantum superposition to explore
        multiple solutions simultaneously.
        
        Your task is to store the QUBO problem instance and set up the basic QAOA parameters.
        You need to determine how many qubits are required (one for each decision variable),
        set the circuit depth parameter p (number of alternating layers), and initialize
        random starting values for the variational parameters gamma and beta that will be
        optimized classically.
        
        Remember that gamma parameters control how much the problem Hamiltonian affects the
        state evolution, while beta parameters control the mixing between different solutions.
        """
        self.qubo = qubo
        self.p = p
        self.n_qubits = qubo.num_variables

        self.gamma = np.random.uniform(0, 2*np.pi, self.p)
        self.beta = np.random.uniform(0, np.pi, self.p)
    
    def create_initial_state(self) -> np.ndarray:
        """
        Create the initial quantum state for QAOA: uniform superposition over all possibilities.
        
        QAOA always starts with a uniform superposition state where every possible solution
        has equal probability amplitude. This is like starting with perfect uncertainty about
        which farm-food allocation is best, allowing the quantum algorithm to explore all
        possibilities simultaneously.
        
        For an n-qubit system, you need to create a state vector with 2^n components, where
        each component represents the amplitude for one possible binary solution. The uniform
        superposition gives each basis state an amplitude of 1/√(2^n) so that the total
        probability (sum of squared amplitudes) equals 1.
        
        This initial state represents the quantum mechanical equivalent of "we have no idea
        which farms should grow which foods yet, so let's consider all possibilities equally."
        """
        
        initial_state = np.ones(2**self.n_qubits, dtype=complex) / np.sqrt(2**self.n_qubits)
        return initial_state
    
    def apply_problem_hamiltonian(self, state: np.ndarray, gamma: float) -> np.ndarray:
        """
        Apply the problem Hamiltonian evolution: e^(-iγH_p) where H_p encodes the QUBO objective.
        
        The problem Hamiltonian encodes your optimization objective and constraints as quantum
        phases. When applied to the quantum state, it rotates the phase of each basis state
        by an amount proportional to how good or bad that solution is. Better solutions get
        phases that constructively interfere, while worse solutions get phases that lead to
        destructive interference.
        
        Your task is to implement the phase evolution for both diagonal terms (single-qubit
        phases) and off-diagonal terms (two-qubit interaction phases) from the QUBO matrix.
        For diagonal terms, apply a phase exp(-iγ * Q[i,i]) when qubit i is in state |1⟩.
        For off-diagonal terms, apply a phase exp(-iγ * Q[i,j]) when both qubits i and j are
        in state |1⟩.
        
        Think of this as teaching the quantum state which solutions are better by giving them
        favorable phase relationships that will lead to higher measurement probabilities after
        the full QAOA circuit.
        """
        new_state = np.copy(state)

        for i in range(self.n_qubits):
            phase_coeff = self.qubo.Q[i,i]

            for bitstring in range(2**self.n_qubits):
                if (bitstring >> i) & 1:
                    new_state[bitstring] *= np.exp(-1j * gamma * phase_coeff)

        for i in range(self.n_qubits):
            for j in range(i+1, self.n_qubits):
                if abs(self.qubo.Q[i,j]) > 1e-10:
                    phase_coeff = self.qubo.Q[i,j]

                    for bitsring in range(2**self.n_qubits):
                        bit_i = (bitstring >> i) & 1
                        bit_j = (bitstring >> j) & 1

                        if bit_i == 1 and bit_j == 1:  # Both qubits are |1⟩
                            new_state[bitstring] *= np.exp(-1j * gamma * phase_coeff)


        return new_state
    
    def apply_mixer_hamiltonian(self, state: np.ndarray, beta: float) -> np.ndarray:
        """
        Apply the mixer Hamiltonian: e^(-iβH_m) where H_m performs X rotations on all qubits.
        
        The mixer Hamiltonian allows the quantum state to explore different solutions by
        creating superpositions between states that differ by flipping individual qubits.
        This is like allowing each farm-food decision to be changed while maintaining the
        quantum superposition across all possibilities.
        
        Mathematically, this applies simultaneous X rotations to all qubits. For each
        computational basis state, the mixer creates amplitudes for states where individual
        qubits are flipped. The rotation angle β controls how much mixing occurs - small β
        means little exploration, large β means lots of exploration but potentially losing
        the phase information from the problem Hamiltonian.
        
        Your implementation should create a new state where each basis state contributes
        to multiple other basis states through X rotations. Think of this as the quantum
        equivalent of making small random changes to a classical solution to explore the
        neighborhood.
        """
        state = state.astype(complex)
        new_state = np.zeros_like(state, dtype=complex)
        
        cos_beta = np.cos(beta)
        sin_beta = np.sin(beta)
        
        # For each computational basis state |x⟩
        for bitstring in range(2**self.n_qubits):
            # For each possible subset of qubits to flip
            for flip_mask in range(2**self.n_qubits):
                # Count number of bit flips
                n_flips = bin(flip_mask).count('1')
                
                # Target bitstring after flipping
                target_bitstring = bitstring ^ flip_mask
                
                # Amplitude: cos^(n-k) * (-i*sin)^k where k = number of flips
                amplitude = (cos_beta**(self.n_qubits - n_flips)) * ((-1j * sin_beta)**n_flips)
                
                new_state[target_bitstring] += amplitude * state[bitstring]
        
        # Verify normalization (should print 1.0)
        normalization = np.sum(np.abs(new_state)**2)
        print(f"Normalization: {normalization:.10f}")
        
        return new_state
    
    def evolve_state(self, gamma_list: List[float], beta_list: List[float]) -> np.ndarray:
        """
        Evolve the initial state through the complete QAOA circuit.
        
        This method implements the full QAOA algorithm by applying alternating layers of
        problem and mixer Hamiltonians. The algorithm starts with the uniform superposition
        initial state, then repeatedly applies the problem Hamiltonian (to bias towards
        good solutions) followed by the mixer Hamiltonian (to explore nearby solutions).
        
        The number of layers p determines how many times this alternation happens. More layers
        generally allow for better solutions but require more parameters to optimize and deeper
        quantum circuits to implement. For each layer, you use the corresponding gamma and beta
        parameters from the input lists.
        
        Your task is to implement this sequential application of Hamiltonians, carefully
        maintaining the quantum state throughout the evolution. The final state should contain
        amplitudes that favor better solutions to your food production optimization problem.
        """
        state = self.create_initial_state()
        
        for i in range(self.p):
            # Apply problem Hamiltonian
            state = self.apply_problem_hamiltonian(state, gamma_list[i])
            # Apply mixer Hamiltonian  
            state = self.apply_mixer_hamiltonian(state, beta_list[i])
        
        return state
    
    def compute_expectation(self, gamma_list: List[float], beta_list: List[float]) -> float:
        """
        Compute the expected value of the problem Hamiltonian for given QAOA parameters.
        
        The expectation value tells us how good the solutions are that we expect to measure
        from the QAOA state on average. This is what we'll optimize classically to find the
        best QAOA parameters. A lower expectation value means we're more likely to measure
        high-quality solutions.
        
        To compute this, you need to evolve the quantum state through the QAOA circuit,
        then calculate the weighted average of solution qualities, where the weights are
        the measurement probabilities (squared amplitudes) and the qualities are the QUBO
        objective values for each possible solution.
        
        This expectation value is the bridge between the quantum world (where we have
        superpositions and amplitudes) and the classical world (where we need concrete
        numbers to optimize). Your classical optimizer will try different gamma and beta
        values to minimize this expectation.
        """
        state = self.evolve_state(gamma_list, beta_list)
        probabilities = np.abs(state)**2
        
        expectation = 0.0
        for bitstring in range(2**self.n_qubits):
            # Convert bitstring to binary solution
            solution = np.array([(bitstring >> i) & 1 for i in range(self.n_qubits)])
            
            # Calculate QUBO objective for this solution
            objective_value = self.qubo.evaluate_solution(solution)
            expectation += probabilities[bitstring] * objective_value
        
        return expectation
    
    def sample_solutions(self, gamma_list: List[float], beta_list: List[float], 
                        n_shots: int = 1000) -> List[np.ndarray]:
        """
        Sample concrete solutions from the QAOA state to see what allocations are recommended.
        
        After finding good QAOA parameters, you want to actually get recommended farm-food
        allocations that you can implement in practice. This method simulates measuring the
        quantum state multiple times to see which solutions appear most frequently.
        
        Each "shot" is like running the quantum algorithm once and measuring all qubits to
        get a concrete binary solution. Solutions with higher amplitudes in the quantum state
        will appear more frequently in your samples. This gives you a distribution of
        recommended solutions rather than just one answer.
        
        Your implementation should calculate measurement probabilities from the evolved quantum
        state, then randomly sample basis states according to these probabilities. Each sampled
        basis state corresponds to a specific pattern of which farms grow which foods.
        """
        state = self.evolve_state(gamma_list, beta_list)
        probabilities = np.abs(state)**2
        
        
        # Sample bitstrings
        sampled_bitstrings = np.random.choice(
            2**self.n_qubits, size=n_shots, p=probabilities
        )
        
        # Convert to binary solutions
        solutions = []
        for bitstring in sampled_bitstrings:
            solution = np.array([(bitstring >> i) & 1 for i in range(self.n_qubits)])
            solutions.append(solution)
        
        return solutions

# Student Task: Test your QAOA implementation
print("\nTesting Food Production QAOA")
print("============================")

"""
Your task: Create and test a QAOA instance to solve your food production problem.

1. Create a QAOA instance:
   - Use the QUBO problem you created in the previous section
   - Start with a simple circuit depth (p=1) for initial testing
   - Print initialization details to confirm setup

2. Test expectation value computation:
   - Choose some test parameters for gamma and beta (try values around 0.5)
   - Compute the expectation value to verify your implementation works
   - Print the result to see if it gives reasonable values

3. Sample solutions from the QAOA state:
   - Use your test parameters to sample solutions (try 50-100 shots)
   - Print how many solutions you got to verify sampling works

4. Analyze a few sample solutions:
   - For the first few solutions sampled:
     - Calculate the QUBO objective value using your evaluate_solution method
     - Get a human-readable description using get_solution_description
     - Print both the objective value and the farm-food allocation description
   - This will help you understand what kinds of solutions your QAOA is finding

5. Optional: Try different parameter values and see how the solutions change

This testing will confirm that your QAOA implementation can actually solve food production
optimization problems and give you insight into the types of solutions it recommends!
"""

# Write your code here to test the QAOA implementation:
qaoa = FoodProductionQAOA(qubo, p=1)

# Test expectation computation
test_gamma = [0.5]
test_beta = [0.3]
test_expectation = qaoa.compute_expectation(test_gamma, test_beta)
print(f"Test expectation value: {test_expectation:.4f}")

# Sample some solutions
sample_solutions = qaoa.sample_solutions(test_gamma, test_beta, n_shots=100)
print(f"Sampled {len(sample_solutions)} solutions")

# Show a few sample solutions
for i, solution in enumerate(sample_solutions[:3]):
    objective = qubo.evaluate_solution(solution)
    description = qubo.get_solution_description(solution)
    print(f"\nSample {i+1} (objective: {objective:.4f}):")
    print(description if description else "No allocations")


Testing Food Production QAOA
Normalization: 1.0000000000
Test expectation value: -4.4003
Normalization: 1.0000000000
Sampled 100 solutions

Sample 1 (objective: -27.4950):
North_Farm grows Rice, South_Farm grows Rice, East_Farm grows Wheat

Sample 2 (objective: -21.1900):
North_Farm grows Soybeans, South_Farm grows Wheat

Sample 3 (objective: -14.4750):
North_Farm grows Rice


## 3. Parameter Optimization for Food Production QAOA

The key to successful QAOA is finding optimal parameters γ and β. Let's implement the optimization strategies used in the OQI_Project.

### Optimization Challenges in Food Production:
1. **Multi-modal landscape**: Multiple local optima due to discrete constraints
2. **Constraint penalties**: Parameter space affected by penalty weights
3. **Problem scaling**: Optimization difficulty grows with farm/food count


Your comprehensive task: Implement the complete parameter optimization system.

1. Complete the optimize_qaoa_parameters function:
   - Design an objective function that extracts gamma and beta parameters from a flattened vector
   - Use the QAOA's compute_expectation method to evaluate parameter quality
   - Implement proper bounds on parameters (gamma and beta typically range from 0 to 2π)
   - Create a callback system to track optimization progress
   - Use scipy.optimize.minimize with your chosen method
   - Return optimal parameters, best expectation value, and convergence history

2. Build the comparison framework:
   - Test multiple optimization methods: COBYLA, Nelder-Mead, Powell, and optionally others
   - For each method, measure both solution quality and computation time
   - Handle cases where optimization might fail or get stuck
   - Create a statistical summary comparing method performance

3. Test your optimization system:
   - Run parameter optimization on your test QAOA instance
   - Use verbose output to observe how the optimization progresses
   - Compare the initial random parameters with the optimized ones
   - Calculate how much the expectation value improved

4. Analyze optimization landscapes:
   - Try multiple random starting points to see if you find different local optima
   - Plot the convergence curves to understand optimization dynamics
   - Investigate how sensitive the solution is to the starting parameters

5. Connect to real-world insights:
   - Consider what the optimal parameters tell you about the problem structure
   - Think about how the optimization landscape might change with different constraints
   - Reflect on computational trade-offs between optimization quality and time

This implementation will be the foundation for making QAOA practical for real food production
optimization problems, where finding good parameters efficiently is crucial for success!
"""

# Write your parameter optimization implementation here:

In [ ]:

def optimize_qaoa_parameters(qaoa: FoodProductionQAOA, 
                           method: str = 'COBYLA',
                           max_iterations: int = 100,
                           verbose: bool = True) -> Tuple[np.ndarray, float]:
    """
    Optimize QAOA parameters using classical optimization methods.
    
    This function implements the hybrid quantum-classical approach that is at the heart of QAOA.
    The quantum circuit provides expectation values for different parameter settings, while
    classical optimization algorithms search for the parameter values that give the best
    expected performance.
    
    The challenge is that the parameter landscape can be quite complex with multiple local
    optima, especially for constrained optimization problems like food production. Different
    classical optimizers have different strengths - some are better at avoiding local optima,
    others are faster for smooth landscapes.
    
    Your task is to implement the optimization loop that tries different gamma and beta values,
    evaluates the quantum expectation for each, and uses classical optimization methods to
    find the best parameters. You'll also want to track the optimization progress to understand
    how the algorithm is converging.
    
    Think about how to:
    1. Structure the objective function that the classical optimizer will minimize
    2. Convert between the QAOA's separate gamma/beta arrays and the flattened parameter vector needed by optimizers
    3. Implement a callback function to monitor optimization progress
    4. Handle the fact that QAOA expectation values can be noisy
    5. Return both the optimal parameters and the optimization history for analysis
    
    Remember that classical optimizers like COBYLA, Nelder-Mead, and Powell have different
    characteristics. COBYLA handles bound constraints well, Nelder-Mead is robust for noisy
    functions, and Powell is efficient for smooth landscapes.
    """
    pass

def compare_optimization_methods(qaoa: FoodProductionQAOA) -> Dict[str, Dict]:
    """
    Compare different classical optimization methods for QAOA parameter optimization.
    
    Different optimization algorithms have different characteristics that make them more or
    less suitable for QAOA parameter optimization. Some are good at global search, others
    are fast for local optimization. Some handle constraints well, others are better for
    unconstrained problems.
    
    Your task is to test several optimization methods on the same QAOA problem and compare
    their performance in terms of solution quality, computation time, and reliability.
    This will help you understand which methods work best for food production optimization.
    
    Consider implementing:
    1. A systematic comparison across multiple optimization methods
    2. Timing and performance tracking for each method
    3. Statistical analysis of solution quality differences
    4. Error handling for methods that might fail to converge
    5. A summary analysis that recommends the best method for different scenarios
    
    The comparison should reveal insights about:
    - Which methods find better parameter values consistently
    - How computation time scales with problem complexity
    - Which methods are most robust to different problem structures
    - Whether any methods get stuck in poor local optima
    """
    pass

# Student Task: Implement and test parameter optimization
print("Parameter Optimization for Food Production QAOA")
print("==============================================")


In [ ]:
def optimize_qaoa_parameters(qaoa: FoodProductionQAOA, 
                           method: str = 'COBYLA',
                           max_iterations: int = 100,
                           verbose: bool = True) -> Tuple[np.ndarray, float]:
    """
    Optimize QAOA parameters using classical optimization methods.
    
    This function implements the hybrid quantum-classical approach that is at the heart of QAOA.
    The quantum circuit provides expectation values for different parameter settings, while
    classical optimization algorithms search for the parameter values that give the best
    expected performance.
    
    The challenge is that the parameter landscape can be quite complex with multiple local
    optima, especially for constrained optimization problems like food production. Different
    classical optimizers have different strengths - some are better at avoiding local optima,
    others are faster for smooth landscapes.
    
    Your task is to implement the optimization loop that tries different gamma and beta values,
    evaluates the quantum expectation for each, and uses classical optimization methods to
    find the best parameters. You'll also want to track the optimization progress to understand
    how the algorithm is converging.
    """
    
    def objective_function(params):
        """The function that classical optimizers will try to minimize."""
        # Extract gamma and beta parameters from the flattened parameter vector
        # Remember that we want to minimize the expectation value (better solutions have lower QUBO values)
        # But optimization algorithms minimize, so we return the negative expectation if we want to maximize
        pass
    
    # Set up initial parameters and optimization tracking
    # Combine gamma and beta parameters into a single vector for the optimizer
    # Initialize tracking lists to monitor optimization progress
    
    if verbose:
        # Print initial status and parameters
        pass
    
    def callback(params):
        """Function called after each optimization iteration to track progress."""
        # Calculate objective value and add to history
        # Print periodic updates if verbose mode is enabled
        pass
    
    # Run the classical optimization
    # Use scipy.optimize.minimize with the specified method
    # Configure maximum iterations and callback function
    # Time the optimization process
    
    # Extract and return results
    # Get optimal parameters from the optimization result
    # Calculate the final optimal value
    # Return parameters, objective value, and optimization history
    
    if verbose:
        # Print final optimization results including time taken and success status
        pass
    
    pass

def compare_optimization_methods(qaoa: FoodProductionQAOA) -> Dict[str, Dict]:
    """
    Compare different classical optimization methods for QAOA parameter optimization.
    
    Different optimization algorithms have different characteristics that make them more or
    less suitable for QAOA parameter optimization. Some are good at global search, others
    are fast for local optimization. Some handle constraints well, others are better for
    unconstrained problems.
    
    Your task is to test several optimization methods on the same QAOA problem and compare
    their performance in terms of solution quality, computation time, and reliability.
    This will help you understand which methods work best for food production optimization.
    """
    methods = ['COBYLA', 'Nelder-Mead', 'Powell']
    results = {}
    
    print("Comparing Parameter Optimization Methods")
    print("======================================")
    
    # For each optimization method:
    # - Run the parameter optimization using that method
    # - Time how long it takes
    # - Record the optimal parameters and objective value
    # - Handle any exceptions that might occur
    # - Store all results for comparison
    
    # Print a summary comparing the performance of different methods
    
    return results

# Student Task: Optimize parameters for your food production problem
print("Parameter Optimization for Food Production QAOA")
print("==============================================")

"""
Your task: Implement and run parameter optimization for your QAOA instance.

1. Complete the optimize_qaoa_parameters function:
   - Implement the objective function that extracts gamma/beta and computes expectation
   - Set up initial parameters by combining the QAOA instance's gamma and beta arrays
   - Use scipy.optimize.minimize to run the optimization
   - Implement the callback function to track progress
   - Return optimal parameters, best value, and optimization history

2. Test the optimization:
   - Run optimize_qaoa_parameters on your QAOA instance
   - Use verbose=True to see the optimization progress
   - Store the results (optimal parameters, best value, optimization history)

3. Update your QAOA instance:
   - Set the gamma and beta parameters of your QAOA instance to the optimal values
   - This will be used for subsequent analysis

4. Compare optimization methods:
   - Complete the compare_optimization_methods function
   - Test COBYLA, Nelder-Mead, and Powell methods
   - Compare their performance in terms of solution quality and computation time

5. Visualize the optimization convergence:
   - Create a plot showing how the expectation value improves over iterations
   - Use matplotlib to plot the optimization history
   - Add proper labels and formatting

6. Calculate improvement:
   - Compare the initial and final expectation values
   - Calculate the percentage improvement achieved through optimization

This will show you how classical optimization can dramatically improve the performance
of your quantum algorithm by finding the best parameter settings!
"""

# Write your code here to implement and test parameter optimization:

## 4. Recursive QAOA for Large Food Production Problems

For larger problems (many farms and foods), we implement Recursive QAOA (RQAOA) as used in the OQI_Project's `optimize_with_recursive_qaoa_merge` method.

### Recursive QAOA Algorithm:
1. **Run QAOA** on the full problem
2. **Identify confident variables** (high measurement probability)
3. **Fix confident variables** to their most likely values
4. **Create reduced problem** with remaining variables
5. **Repeat recursively** until problem is small enough

### Benefits for Food Production:
- **Scalability**: Handle 10+ farms with 5+ foods each
- **Better solutions**: Often finds better allocations than standard QAOA
- **Interpretability**: Shows which farm-food decisions are most certain

---

In [ ]:
class RecursiveFoodQAOA:
    """
    Recursive QAOA for large food production problems - educational version based on OQI_Project approach.
    
    Recursive QAOA is a powerful technique for scaling quantum optimization to larger problems
    by intelligently breaking them into smaller, more manageable pieces. This approach is used
    in the OQI_Project's `optimize_with_recursive_qaoa_merge` method for handling complex
    agricultural optimization scenarios.
    
    The core insight is that after running QAOA on a large problem, some variables often have
    very high confidence in their optimal values (close to 0 or 1 probability). These confident
    variables can be "fixed" to their most likely values, creating a smaller problem that's
    easier to solve optimally.
    """
    
    def __init__(self, qubo: FoodProductionQUBO, 
                 confidence_threshold: float = 0.8,
                 min_problem_size: int = 4,
                 max_recursion_depth: int = 5):
        """
        Initialize the Recursive QAOA system.
        
        Your task is to set up the framework for recursive problem solving. Think about how
        you'll track which variables have been fixed, maintain the recursion history for
        analysis, and configure the stopping criteria that determine when to stop recursing.
        
        The confidence threshold determines how certain a variable must be before we fix it.
        The minimum problem size sets when we switch to exact solution methods. The maximum
        recursion depth prevents infinite loops in pathological cases.
        """
        self.original_qubo = qubo
        self.confidence_threshold = confidence_threshold
        self.min_problem_size = min_problem_size
        self.max_recursion_depth = max_recursion_depth
        
        # Track fixed variables and their values
        self.fixed_variables = {}  # var_idx -> value
        self.recursion_history = []
        
    def measure_variable_confidence(self, qaoa: FoodProductionQAOA, 
                                  gamma_list: List[float], beta_list: List[float]) -> Dict[int, float]:
        """
        Measure how confident we are about each variable's optimal value.
        
        After running QAOA, the quantum state contains information about which solutions are
        most likely. Some variables might be very likely to be 0 or 1 across all good solutions,
        indicating high confidence. Others might be close to 50/50, indicating uncertainty.
        
        Your implementation should:
        1. Evolve the QAOA state with the given parameters
        2. Calculate the probability that each variable is 1 by summing over all basis states
        3. Convert these probabilities to confidence measures (distance from maximum uncertainty)
        4. Return a mapping from variable index to confidence level
        
        High confidence variables are candidates for fixing in the recursive process.
        """
        state = qaoa.evolve_state(gamma_list, beta_list)
        probabilities = np.abs(state)**2
        
        variable_confidence = {}
        
        for var_idx in range(qaoa.n_qubits):
            prob_one = 0.0
            
            # Sum probabilities where variable var_idx is |1⟩
            for bitstring in range(2**qaoa.n_qubits):
                if (bitstring >> var_idx) & 1:
                    prob_one += probabilities[bitstring]
            
            # Confidence is how far from 0.5 (maximum uncertainty)
            confidence = abs(prob_one - 0.5) * 2
            variable_confidence[var_idx] = confidence
        
        return variable_confidence
    
    def create_reduced_qubo(self, original_qubo: FoodProductionQUBO, 
                           fixed_vars: Dict[int, int]) -> Tuple[FoodProductionQUBO, Dict[int, int]]:
        """
        Create a smaller QUBO problem by fixing confident variables to their optimal values.
        
        This is the heart of the recursive approach. You need to take a large QUBO matrix
        and create a smaller one that represents the same optimization problem but with some
        variables already decided.
        
        Your implementation should:
        1. Identify which variables remain unfixed and need new variable indices
        2. Create a mapping between old and new variable indices
        3. Extract the relevant farms and foods for the reduced problem
        4. Copy the appropriate parts of the QUBO matrix for unfixed variables
        5. Adjust the objective function to account for the interactions with fixed variables
        6. Maintain the mathematical correctness of the optimization problem
        
        This requires careful attention to the QUBO matrix structure and how fixing variables
        affects both the objective function and constraint penalty terms.
        """
        
        # Identify remaining (unfixed) variables
        all_vars = set(range(original_qubo.n_variables))
        fixed_var_set = set(fixed_vars.keys())
        remaining_vars = sorted(all_vars - fixed_var_set)
        
        if len(remaining_vars) == 0:
            return None, {}
        
        # Create mapping from old to new variable indices
        old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(remaining_vars)}
        
        # Extract farms and foods for remaining variables
        remaining_farms = set()
        remaining_foods = set()
        
        for var_idx in remaining_vars:
            farm_idx, food_idx = original_qubo.get_farm_food_from_index(var_idx)
            remaining_farms.add(farm_idx)
            remaining_foods.add(food_idx)
        
        # Create reduced problem
        reduced_farms = [original_qubo.farms[i] for i in sorted(remaining_farms)]
        reduced_foods = {name: data for name, data in original_qubo.foods.items() 
                        if original_qubo.food_names.index(name) in remaining_foods}
        
        reduced_qubo = FoodProductionQUBO(reduced_farms, reduced_foods)
        
        # Copy relevant parts of Q matrix
        for i, old_i in enumerate(remaining_vars):
            for j, old_j in enumerate(remaining_vars):
                reduced_qubo.Q[i, j] = original_qubo.Q[old_i, old_j]
        
        # Adjust for fixed variables (modify diagonal terms)
        for var_idx in remaining_vars:
            new_idx = old_to_new[var_idx]
            
            # Add contributions from fixed variables
            for fixed_var, fixed_value in fixed_vars.items():
                if fixed_var != var_idx:
                    # Add interaction term contribution
                    interaction = original_qubo.Q[min(var_idx, fixed_var), max(var_idx, fixed_var)]
                    reduced_qubo.Q[new_idx, new_idx] += interaction * fixed_value
        
        return reduced_qubo, old_to_new
    
    def solve_recursive(self, current_qubo: FoodProductionQUBO, 
                       depth: int = 0) -> Tuple[Dict[int, int], float]:
        """
        Recursively solve the QUBO problem using the divide-and-conquer approach.
        
        This method implements the main recursive algorithm that alternates between QAOA
        optimization and problem reduction until reaching a base case that can be solved
        exactly or with simple methods.
        
        Your implementation should:
        1. Check termination conditions (maximum depth, minimum problem size)
        2. Run QAOA on the current problem size
        3. Measure variable confidence from the QAOA results
        4. Identify and fix variables that exceed the confidence threshold
        5. Create a reduced problem with the confident variables fixed
        6. Recursively solve the reduced problem
        7. Combine the fixed variables with the recursive solution
        8. Track the recursion history for analysis and debugging
        
        Consider how to handle edge cases like no confident variables found, or when the
        reduced problem becomes empty. Also think about how to merge solutions from different
        recursion levels while maintaining variable index consistency.
        """
        
        if depth > self.max_recursion_depth:
            print(f"Maximum recursion depth reached at {depth}")
            return {}, float('inf')
        
        if current_qubo.n_variables <= self.min_problem_size:
            print(f"Reached minimum problem size: {current_qubo.n_variables} variables")
            # Solve small problem exactly or with simple QAOA
            return self.solve_small_problem(current_qubo)
        
        print(f"Recursion depth {depth}: solving {current_qubo.n_variables} variables")
        
        # Run QAOA on current problem
        qaoa = FoodProductionQAOA(current_qubo, p=1)
        optimal_params, optimal_value, _ = optimize_qaoa_parameters(qaoa, verbose=False)
        
        # Measure variable confidence
        gamma_list = optimal_params[:qaoa.p]
        beta_list = optimal_params[qaoa.p:]
        confidence = self.measure_variable_confidence(qaoa, gamma_list, beta_list)
        
        # Find confident variables
        confident_vars = {}
        for var_idx, conf in confidence.items():
            if conf > self.confidence_threshold:
                # Determine most likely value
                state = qaoa.evolve_state(gamma_list, beta_list)
                probabilities = np.abs(state)**2
                
                prob_one = sum(probabilities[bs] for bs in range(2**qaoa.n_qubits) 
                              if (bs >> var_idx) & 1)
                
                confident_vars[var_idx] = 1 if prob_one > 0.5 else 0
        
        if len(confident_vars) == 0:
            print(f"No confident variables found at depth {depth}")
            # Return best solution from current QAOA
            solutions = qaoa.sample_solutions(gamma_list, beta_list, n_shots=100)
            best_solution = min(solutions, key=lambda sol: current_qubo.evaluate_solution(sol))
            solution_dict = {i: int(best_solution[i]) for i in range(len(best_solution))}
            return solution_dict, current_qubo.evaluate_solution(best_solution)
        
        print(f"Fixed {len(confident_vars)} confident variables")
        self.recursion_history.append({
            'depth': depth,
            'problem_size': current_qubo.n_variables,
            'fixed_count': len(confident_vars),
            'confident_vars': confident_vars.copy()
        })
        
        # Create reduced problem
        reduced_qubo, var_mapping = self.create_reduced_qubo(current_qubo, confident_vars)
        
        if reduced_qubo is None:
            return confident_vars, optimal_value
        
        # Solve reduced problem recursively
        reduced_solution, reduced_objective = self.solve_recursive(reduced_qubo, depth + 1)
        
        # Combine solutions
        full_solution = confident_vars.copy()
        for new_idx, value in reduced_solution.items():
            # Map back to original indices
            for old_idx, mapped_idx in var_mapping.items():
                if mapped_idx == new_idx:
                    full_solution[old_idx] = value
                    break
        
        return full_solution, reduced_objective
    
    def solve_small_problem(self, qubo: FoodProductionQUBO) -> Tuple[Dict[int, int], float]:
        """
        Solve small problems using exact methods or optimized QAOA.
        
        When the recursive process reaches a small enough problem size, it's often more
        efficient to use exact enumeration or highly optimized QAOA rather than continuing
        the recursive decomposition.
        
        Your implementation should:
        1. For very small problems (≤10 variables): Use exhaustive search over all 2^n solutions
        2. For slightly larger problems: Use QAOA with more sophisticated parameter optimization
        3. Handle the transition between exact and approximate methods gracefully
        4. Return solutions in the same format as the recursive method for consistency
        
        This method represents the "base case" of your recursive algorithm and should guarantee
        high-quality solutions for the reduced subproblems.
        """
        if qubo.n_variables <= 10:
            # Exhaustive search for very small problems
            best_solution = None
            best_objective = float('inf')
            
            for bitstring in range(2**qubo.n_variables):
                solution = np.array([(bitstring >> i) & 1 for i in range(qubo.n_variables)])
                objective = qubo.evaluate_solution(solution)
                
                if objective < best_objective:
                    best_objective = objective
                    best_solution = solution
            
            solution_dict = {i: int(best_solution[i]) for i in range(len(best_solution))}
            return solution_dict, best_objective
        
        else:
            # Use QAOA for larger small problems
            qaoa = FoodProductionQAOA(qubo, p=1)
            optimal_params, _, _ = optimize_qaoa_parameters(qaoa, verbose=False)
            
            gamma_list = optimal_params[:qaoa.p]
            beta_list = optimal_params[qaoa.p:]
            solutions = qaoa.sample_solutions(gamma_list, beta_list, n_shots=100)
            
            best_solution = min(solutions, key=lambda sol: qubo.evaluate_solution(sol))
            solution_dict = {i: int(best_solution[i]) for i in range(len(best_solution))}
            
            return solution_dict, qubo.evaluate_solution(best_solution)

# Test Recursive QAOA
print("Testing Recursive QAOA for Food Production")
print("=========================================")

# Create a larger problem for testing
larger_farms = ['Farm_A', 'Farm_B', 'Farm_C', 'Farm_D']
larger_foods = {name: data for name, data in list(SAMPLE_FOODS.items())[:4]}

large_qubo = FoodProductionQUBO(larger_farms, larger_foods)
large_qubo.set_multi_objective_weights(sustainability_weights)

# Add constraints
large_capacities = {farm: 2 for farm in larger_farms}
large_qubo.add_farm_capacity_constraints(large_capacities, penalty_weight=3.0)
large_qubo.add_diversity_constraints(min_farms_per_food=1, penalty_weight=2.0)

print(f"Large problem: {large_qubo.F} farms × {large_qubo.C} foods = {large_qubo.n_variables} variables")

# Solve with Recursive QAOA
recursive_qaoa = RecursiveFoodQAOA(large_qubo, confidence_threshold=0.7, min_problem_size=6)
solution, objective = recursive_qaoa.solve_recursive(large_qubo)

print(f"\nRecursive QAOA Solution:")
print(f"Objective value: {objective:.6f}")
print(f"Solution: {solution}")

# Convert to readable format
solution_array = np.zeros(large_qubo.n_variables)
for var_idx, value in solution.items():
    solution_array[var_idx] = value

description = large_qubo.get_solution_description(solution_array)
print(f"\nSolution description:")
print(description if description else "No allocations")

print(f"\nRecursion history:")
for step in recursive_qaoa.recursion_history:
    print(f"Depth {step['depth']}: {step['problem_size']} vars → fixed {step['fixed_count']}")

## 5. Performance Analysis: Quantum vs Classical

Let's compare our QAOA implementations with classical approaches for food production optimization.

### Comparison Methods:
1. **Standard QAOA**: Basic quantum approach
2. **Recursive QAOA**: Quantum divide-and-conquer
3. **Random Search**: Classical baseline
4. **Greedy Algorithm**: Classical heuristic
5. **Simulated Annealing**: Classical metaheuristic

---

In [ ]:
def classical_random_search(qubo: FoodProductionQUBO, n_trials: int = 1000) -> Tuple[np.ndarray, float]:
    """
    Implement a baseline random search algorithm for comparison with quantum methods.
    
    Random search provides a simple baseline that helps us understand whether more sophisticated
    algorithms (quantum or classical) are actually providing value. It works by randomly
    generating many candidate solutions and keeping the best one found.
    
    Your task is to implement this baseline method that:
    1. Generates random binary solutions for the farm-food allocation problem
    2. Evaluates each solution using the QUBO objective function
    3. Tracks the best solution found across all trials
    4. Returns both the best solution and its objective value
    
    This will serve as a reality check for more sophisticated methods - if they can't beat
    random search by a significant margin, they're probably not adding much value!
    """
    best_solution = None
    best_objective = float('inf')
    
    for _ in range(n_trials):
        # Random binary solution
        solution = np.random.randint(0, 2, size=qubo.n_variables)
        objective = qubo.evaluate_solution(solution)
        
        if objective < best_objective:
            best_objective = objective
            best_solution = solution
    
    return best_solution, best_objective

def classical_greedy_algorithm(qubo: FoodProductionQUBO) -> Tuple[np.ndarray, float]:
    """
    Implement a greedy heuristic that makes locally optimal choices.
    
    Greedy algorithms make the best available choice at each step without considering
    global optimization. For food production, this might mean always choosing the
    farm-food combination with the highest individual benefit.
    
    Your implementation should:
    1. Calculate the individual benefit of each possible farm-food combination
    2. Sort these combinations by their standalone attractiveness
    3. Greedily select combinations while respecting constraints
    4. Handle constraint violations by checking feasibility before adding new selections
    
    Think about how to handle the fact that the greedy choice at each step might lead
    to constraint violations or suboptimal global solutions. Consider implementing
    simple constraint checking to ensure the greedy choices remain feasible.
    """
    solution = np.zeros(qubo.n_variables, dtype=int)
    
    # Calculate individual variable benefits
    variable_benefits = []
    for var_idx in range(qubo.n_variables):
        # Benefit of setting this variable to 1
        benefit = -qubo.Q[var_idx, var_idx]  # Negative because QUBO minimizes
        variable_benefits.append((benefit, var_idx))
    
    # Sort by benefit (highest first)
    variable_benefits.sort(reverse=True)
    
    # Greedily select variables while respecting constraints
    for benefit, var_idx in variable_benefits:
        # Try setting this variable to 1
        test_solution = solution.copy()
        test_solution[var_idx] = 1
        
        # Check if this improves the objective (simple constraint handling)
        current_obj = qubo.evaluate_solution(solution)
        test_obj = qubo.evaluate_solution(test_solution)
        
        if test_obj < current_obj:
            solution[var_idx] = 1
    
    return solution, qubo.evaluate_solution(solution)

def classical_simulated_annealing(qubo: FoodProductionQUBO, 
                                initial_temp: float = 1.0,
                                cooling_rate: float = 0.95,
                                max_iterations: int = 1000) -> Tuple[np.ndarray, float]:
    """
    Implement simulated annealing as a sophisticated classical baseline.
    
    Simulated annealing is a probabilistic optimization technique that starts with high
    randomness (temperature) and gradually becomes more selective as it cools down.
    This allows it to escape local optima early on while converging to good solutions.
    
    Your implementation should:
    1. Start with a random solution and set the initial temperature
    2. In each iteration, generate a neighbor solution by flipping one variable
    3. Accept improvements immediately, and accept worse solutions with probability exp(-ΔE/T)
    4. Gradually reduce the temperature according to the cooling schedule
    5. Track the best solution found throughout the process
    
    Consider how to choose good neighbors (which variables to flip), how to set the
    temperature schedule, and how to balance exploration vs. exploitation as the
    algorithm progresses.
    """
    # Start with random solution
    current_solution = np.random.randint(0, 2, size=qubo.n_variables)
    current_objective = qubo.evaluate_solution(current_solution)
    
    best_solution = current_solution.copy()
    best_objective = current_objective
    
    temperature = initial_temp
    
    for iteration in range(max_iterations):
        # Generate neighbor solution (flip one random bit)
        neighbor_solution = current_solution.copy()
        flip_idx = np.random.randint(0, qubo.n_variables)
        neighbor_solution[flip_idx] = 1 - neighbor_solution[flip_idx]
        
        neighbor_objective = qubo.evaluate_solution(neighbor_solution)
        
        # Accept or reject neighbor
        delta = neighbor_objective - current_objective
        
        if delta < 0 or np.random.random() < np.exp(-delta / temperature):
            current_solution = neighbor_solution
            current_objective = neighbor_objective
            
            if current_objective < best_objective:
                best_solution = current_solution.copy()
                best_objective = current_objective
        
        # Cool down
        temperature *= cooling_rate
    
    return best_solution, best_objective

def comprehensive_performance_analysis(problem_sizes: List[Tuple[int, int]]) -> Dict:
    """
    Conduct a thorough comparison of quantum and classical methods across different problem sizes.
    
    This function implements a systematic benchmarking study that reveals when and why
    quantum methods provide advantages over classical approaches. It's designed to answer
    key questions about the practical value of QAOA for food production optimization.
    
    Your implementation should:
    1. Create test problems of varying sizes using your QUBO formulation
    2. Apply all available methods (classical and quantum) to each problem
    3. Measure both solution quality and computational efficiency
    4. Handle cases where methods fail or become computationally intractable
    5. Organize results for systematic analysis and visualization
    
    Key aspects to investigate:
    - How does solution quality scale with problem size for different methods?
    - At what problem sizes do quantum methods become advantageous?
    - What's the computational cost trade-off between quantum and classical approaches?
    - How consistent are the methods across different problem instances?
    - Which types of constraint structures favor quantum vs. classical methods?
    
    This analysis will provide evidence-based insights about when practitioners should
    consider using quantum optimization for real-world food production problems.
    """
    results = {}
    
    for n_farms, n_foods in problem_sizes:
        problem_name = f"{n_farms}F_{n_foods}C"
        print(f"\nAnalyzing {problem_name} ({n_farms} farms, {n_foods} foods)")
        
        # Create problem
        farms = [f"Farm_{i}" for i in range(n_farms)]
        foods = {name: data for name, data in list(SAMPLE_FOODS.items())[:n_foods]}
        
        qubo = FoodProductionQUBO(farms, foods)
        qubo.set_multi_objective_weights(sustainability_weights)
        
        capacities = {farm: min(2, n_foods) for farm in farms}
        qubo.add_farm_capacity_constraints(capacities, penalty_weight=3.0)
        qubo.add_diversity_constraints(min_farms_per_food=1, penalty_weight=2.0)
        
        problem_results = {}
        
        # 1. Classical Random Search
        start_time = time.time()
        rs_solution, rs_objective = classical_random_search(qubo, n_trials=500)
        rs_time = time.time() - start_time
        
        problem_results['Random Search'] = {
            'objective': rs_objective,
            'time': rs_time,
            'solution': rs_solution
        }
        
        # 2. Classical Greedy
        start_time = time.time()
        greedy_solution, greedy_objective = classical_greedy_algorithm(qubo)
        greedy_time = time.time() - start_time
        
        problem_results['Greedy'] = {
            'objective': greedy_objective,
            'time': greedy_time,
            'solution': greedy_solution
        }
        
        # 3. Simulated Annealing
        start_time = time.time()
        sa_solution, sa_objective = classical_simulated_annealing(qubo, max_iterations=500)
        sa_time = time.time() - start_time
        
        problem_results['Simulated Annealing'] = {
            'objective': sa_objective,
            'time': sa_time,
            'solution': sa_solution
        }
        
        # 4. Standard QAOA (if problem not too large)
        if n_farms * n_foods <= 10:
            start_time = time.time()
            qaoa = FoodProductionQAOA(qubo, p=1)
            optimal_params, qaoa_objective, _ = optimize_qaoa_parameters(qaoa, verbose=False)
            
            # Sample best solution
            gamma_list = optimal_params[:qaoa.p]
            beta_list = optimal_params[qaoa.p:]
            solutions = qaoa.sample_solutions(gamma_list, beta_list, n_shots=100)
            best_qaoa_solution = min(solutions, key=lambda sol: qubo.evaluate_solution(sol))
            best_qaoa_objective = qubo.evaluate_solution(best_qaoa_solution)
            qaoa_time = time.time() - start_time
            
            problem_results['QAOA'] = {
                'objective': best_qaoa_objective,
                'time': qaoa_time,
                'solution': best_qaoa_solution,
                'expectation': qaoa_objective
            }
        
        # 5. Recursive QAOA (if problem size manageable)
        if n_farms * n_foods <= 16:
            start_time = time.time()
            recursive_qaoa = RecursiveFoodQAOA(qubo, confidence_threshold=0.7)
            rqaoa_solution_dict, rqaoa_objective = recursive_qaoa.solve_recursive(qubo)
            
            # Convert to array format
            rqaoa_solution = np.zeros(qubo.n_variables)
            for var_idx, value in rqaoa_solution_dict.items():
                rqaoa_solution[var_idx] = value
            
            rqaoa_time = time.time() - start_time
            
            problem_results['Recursive QAOA'] = {
                'objective': rqaoa_objective,
                'time': rqaoa_time,
                'solution': rqaoa_solution,
                'recursion_steps': len(recursive_qaoa.recursion_history)
            }
        
        results[problem_name] = problem_results
        
        # Print summary
        print(f"Results for {problem_name}:")
        for method, result in problem_results.items():
            print(f"  {method}: objective={result['objective']:.4f}, time={result['time']:.3f}s")
    
    return results

# Run comprehensive analysis
print("Comprehensive Performance Analysis")
print("=================================")

# Test on different problem sizes
test_problems = [
    (2, 3),  # Small: 6 variables
    (3, 3),  # Medium: 9 variables  
    (3, 4),  # Larger: 12 variables
    (4, 4),  # Large: 16 variables
]

performance_results = comprehensive_performance_analysis(test_problems)

print("Performance Analysis: Quantum vs Classical Methods")
print("=================================================")

"""
Your comprehensive benchmarking task: Build a complete performance analysis system.

1. Implement the classical baseline methods:
   - Random search: Focus on making it efficient and statistically sound
   - Greedy algorithm: Design heuristics that work well for farm-food allocation
   - Simulated annealing: Tune parameters for good performance on QUBO problems
   - Consider adding other classical methods like genetic algorithms or tabu search

2. Design the benchmarking framework:
   - Create a systematic way to generate test problems of different sizes
   - Implement timing and performance measurement for all methods
   - Handle failures gracefully when methods don't converge or run out of time
   - Store results in a structured format for analysis

3. Test across multiple problem scales:
   - Small problems (6-9 variables): Where exact solutions are possible
   - Medium problems (10-16 variables): Where method differences become apparent
   - Large problems (17+ variables): Where scalability becomes critical
   - Use multiple random instances at each size for statistical significance

4. Measure multiple performance dimensions:
   - Solution quality: How good are the farm-food allocations found?
   - Computational time: How long does each method take?
   - Consistency: How much do results vary across different runs?
   - Scalability: How do performance and time change with problem size?

5. Analyze the results systematically:
   - Identify problem sizes where quantum methods excel
   - Understand what problem characteristics favor different approaches
   - Calculate statistical significance of performance differences
   - Consider practical implications for real agricultural applications

6. Connect to real-world decision making:
   - Think about how practitioners would choose between methods
   - Consider trade-offs between solution quality and computation time
   - Reflect on what types of agricultural problems would benefit most from quantum approaches

This analysis will provide the evidence base for understanding when quantum optimization
can provide practical advantages in agricultural and supply chain optimization!
"""

# Write your performance analysis implementation here:

## 6. Visualization and Analysis

Let's visualize our results to understand when quantum methods provide advantages.

---

In [ ]:
def plot_performance_comparison(results: Dict):
    """
    Create comprehensive visualizations to understand quantum vs classical performance.
    
    Effective data visualization is crucial for understanding when and why quantum methods
    provide advantages. Your visualizations should reveal patterns that aren't obvious
    from raw numbers alone and help practitioners make informed decisions about method selection.
    
    Your task is to create a multi-panel visualization that tells the complete story:
    1. Solution quality comparison: How good are the solutions from each method?
    2. Computational efficiency: How much time does each method require?
    3. Relative performance: How far is each method from the best possible solution?
    4. Consistency analysis: How reliable are the methods across different runs?
    
    Design considerations:
    - Use clear, professional matplotlib plots with proper labels and legends
    - Choose appropriate plot types (line plots for trends, bar plots for comparisons)
    - Include error bars or confidence intervals where appropriate
    - Use color schemes that are accessible and meaningful
    - Annotate significant findings or interesting patterns
    
    The goal is to create visualizations that could appear in a research paper or
    presentation to stakeholders, clearly communicating the practical implications
    of choosing different optimization methods.
    """
    
    # Extract data for plotting
    problem_names = list(results.keys())
    methods = set()
    for problem_result in results.values():
        methods.update(problem_result.keys())
    methods = sorted(list(methods))
    
    # Prepare data
    problem_sizes = [len(name.split('_')[0][:-1]) * len(name.split('_')[1][:-1]) 
                    for name in problem_names]
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Objective values
    for method in methods:
        objectives = []
        for problem_name in problem_names:
            if method in results[problem_name]:
                objectives.append(results[problem_name][method]['objective'])
            else:
                objectives.append(None)
        
        # Filter out None values
        valid_indices = [i for i, obj in enumerate(objectives) if obj is not None]
        valid_sizes = [problem_sizes[i] for i in valid_indices]
        valid_objectives = [objectives[i] for i in valid_indices]
        
        if valid_objectives:
            ax1.plot(valid_sizes, valid_objectives, 'o-', label=method, linewidth=2, markersize=6)
    
    ax1.set_xlabel('Problem Size (Variables)')
    ax1.set_ylabel('Objective Value')
    ax1.set_title('Solution Quality Comparison')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Computation time
    for method in methods:
        times = []
        for problem_name in problem_names:
            if method in results[problem_name]:
                times.append(results[problem_name][method]['time'])
            else:
                times.append(None)
        
        valid_indices = [i for i, t in enumerate(times) if t is not None]
        valid_sizes = [problem_sizes[i] for i in valid_indices]
        valid_times = [times[i] for i in valid_indices]
        
        if valid_times:
            ax2.semilogy(valid_sizes, valid_times, 'o-', label=method, linewidth=2, markersize=6)
    
    ax2.set_xlabel('Problem Size (Variables)')
    ax2.set_ylabel('Computation Time (seconds)')
    ax2.set_title('Computational Efficiency Comparison')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Relative performance (normalized by best solution)
    for problem_name in problem_names:
        problem_result = results[problem_name]
        best_objective = min(result['objective'] for result in problem_result.values())
        
        method_names = list(problem_result.keys())
        relative_performance = [
            (problem_result[method]['objective'] - best_objective) / abs(best_objective)
            for method in method_names
        ]
        
        x_pos = np.arange(len(method_names))
        ax3.bar(x_pos + problem_names.index(problem_name) * 0.15, 
               relative_performance, 
               width=0.15, 
               label=problem_name,
               alpha=0.7)
    
    ax3.set_xlabel('Method')
    ax3.set_ylabel('Relative Performance Gap')
    ax3.set_title('Performance Gap from Best Solution')
    ax3.set_xticks(np.arange(len(methods)))
    ax3.set_xticklabels(methods, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Solution feasibility analysis
    feasibility_data = {}
    for problem_name in problem_names:
        problem_result = results[problem_name]
        
        # Check constraint violations for each method
        for method, result in problem_result.items():
            if method not in feasibility_data:
                feasibility_data[method] = []
            
            # Simple feasibility check: count constraint violations
            # (This is simplified - in practice you'd check specific constraints)
            violations = max(0, result['objective'])  # Positive objective indicates constraint violations
            feasibility_data[method].append(violations)
    
    for method, violations in feasibility_data.items():
        ax4.plot(problem_sizes[:len(violations)], violations, 'o-', label=method, linewidth=2, markersize=6)
    
    ax4.set_xlabel('Problem Size (Variables)')
    ax4.set_ylabel('Constraint Violations')
    ax4.set_title('Solution Feasibility')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def analyze_quantum_advantage(results: Dict):
    """
    Systematically analyze when and why quantum methods provide computational advantages.
    
    Raw performance numbers need interpretation to understand their practical significance.
    This function should provide quantitative analysis of quantum advantage while considering
    the nuances of real-world application.
    
    Your analysis should investigate:
    1. Statistical significance: Are performance differences meaningful or just noise?
    2. Practical significance: Are improvements large enough to matter in practice?
    3. Consistency: Do quantum advantages appear reliably across different problems?
    4. Cost-benefit trade-offs: How do computation time costs compare to solution quality benefits?
    5. Scaling trends: How do advantages change as problems get larger?
    
    Consider both optimistic and pessimistic interpretations of your results. Think about
    what a skeptical classical optimization expert would say, and what a quantum computing
    enthusiast might emphasize. Your analysis should be balanced and evidence-based.
    
    The output should be a narrative analysis that practitioners could use to make
    informed decisions about whether quantum methods are worth considering for their
    specific food production optimization challenges.
    """
    print("\nQuantum Advantage Analysis")
    print("=========================")
    
    for problem_name, problem_result in results.items():
        print(f"\nProblem: {problem_name}")
        
        # Find best classical and quantum solutions
        classical_methods = ['Random Search', 'Greedy', 'Simulated Annealing']
        quantum_methods = ['QAOA', 'Recursive QAOA']
        
        best_classical = float('inf')
        best_quantum = float('inf')
        
        classical_results = {}
        quantum_results = {}
        
        for method, result in problem_result.items():
            if method in classical_methods:
                classical_results[method] = result['objective']
                best_classical = min(best_classical, result['objective'])
            elif method in quantum_methods:
                quantum_results[method] = result['objective']
                best_quantum = min(best_quantum, result['objective'])
        
        if best_quantum < float('inf') and best_classical < float('inf'):
            advantage = (best_classical - best_quantum) / abs(best_classical) * 100
            print(f"  Best classical: {best_classical:.4f}")
            print(f"  Best quantum: {best_quantum:.4f}")
            print(f"  Quantum advantage: {advantage:.2f}%")
            
            if advantage > 5:
                print(f"  → Significant quantum advantage!")
            elif advantage > 0:
                print(f"  → Modest quantum advantage")
            else:
                print(f"  → Classical methods perform better")
        
        # Analyze computational efficiency
        if 'QAOA' in problem_result and 'Simulated Annealing' in problem_result:
            qaoa_time = problem_result['QAOA']['time']
            sa_time = problem_result['Simulated Annealing']['time']
            time_ratio = qaoa_time / sa_time
            print(f"  Time ratio (QAOA/SA): {time_ratio:.2f}x")

# Generate visualizations and analysis
plot_performance_comparison(performance_results)
analyze_quantum_advantage(performance_results)

# Student Task: Create comprehensive analysis and visualization
print("Visualization and Analysis of Results")
print("====================================")

"""
Your comprehensive analysis task: Transform raw performance data into actionable insights.

1. Design the visualization system:
   - Create a multi-panel figure that tells the complete performance story
   - Include solution quality trends, computational efficiency comparisons, and reliability analysis
   - Use appropriate statistical visualizations (box plots, error bars, confidence intervals)
   - Ensure all plots are publication-ready with clear labels and professional formatting
   - Consider interactive elements if working in a Jupyter environment

2. Implement statistical analysis:
   - Calculate statistical significance of performance differences between methods
   - Measure effect sizes to determine practical significance
   - Analyze variance and consistency across multiple runs
   - Identify outliers and investigate their causes
   - Create summary statistics tables for key findings

3. Develop the quantum advantage analysis:
   - Quantify the magnitude of quantum advantages where they exist
   - Identify problem characteristics that favor quantum vs classical methods
   - Analyze computational cost trade-offs (solution quality vs computation time)
   - Investigate scaling trends and extrapolate to larger problem sizes
   - Consider robustness of advantages across different problem instances

4. Create actionable recommendations:
   - Develop decision frameworks for choosing between optimization methods
   - Provide clear guidance on when quantum methods are worth the complexity
   - Address practical considerations like implementation costs and expertise requirements
   - Consider how results might change with different quantum hardware capabilities
   - Connect findings back to real-world agricultural and supply chain applications

5. Validate your analysis:
   - Cross-check findings across different problem sizes and structures
   - Consider alternative interpretations of the data
   - Test the robustness of conclusions to parameter choices
   - Verify that recommendations are supported by statistical evidence

6. Document insights for different audiences:
   - Technical summary for optimization specialists
   - Executive brief for business decision makers
   - Implementation guide for software development teams
   - Research roadmap for continued investigation

This analysis will provide the evidence base for making informed decisions about quantum
optimization adoption in agricultural and supply chain applications!
"""

# Write your visualization and analysis implementation here:

## 7. Key Insights and Next Steps

### 🎯 What We've Learned

1. **QAOA for Food Production**: Successfully implemented QAOA specifically for F×C farm-food optimization problems
2. **Multi-objective QUBO**: Converted complex agricultural constraints into quantum-ready formulations
3. **Recursive Strategies**: Demonstrated how to break large problems into manageable pieces
4. **Performance Trade-offs**: Quantum methods excel on structured problems with specific constraint patterns

### 🔍 Key Findings

- **Problem Size Matters**: QAOA shows advantage on medium-sized problems (8-16 variables)
- **Constraint Structure**: Multi-objective problems benefit from quantum superposition exploration
- **Recursive Approach**: Often finds better solutions than standard QAOA
- **Parameter Optimization**: Critical for QAOA success - poor parameters lead to poor performance

### 🚀 Real-World Applications

This tutorial teaches simplified versions of methods actually used in:
- **Supply Chain Optimization**: Farm-to-processor allocation
- **Resource Management**: Land use optimization under constraints  
- **Sustainability Planning**: Multi-objective agricultural decision making
- **Food Security**: Balancing nutrition, cost, and environmental impact

### 📚 Next Steps

1. **Quantum-Enhanced Benders Decomposition**: Learn the hybrid approach from OQI_Project
2. **Advanced QUBO Techniques**: Handle more complex constraints and objectives
3. **Hardware Implementation**: Adapt for real quantum computers
4. **Error Mitigation**: Deal with noise in quantum computations

### 🔬 Research Directions

- **Larger Problems**: Scale to 50+ farms with 10+ foods
- **Dynamic Optimization**: Handle changing constraints over time
- **Uncertainty**: Incorporate weather and market uncertainty
- **Multi-stakeholder**: Balance competing objectives from different actors

---

**Ready to explore hybrid quantum-classical methods? Continue to the next tutorial on Quantum-Enhanced Benders Decomposition!**

In [ ]:
# Final validation and reflection
def tutorial_reflection_exercise():
    """
    Reflect on what you've learned and validate your implementations.
    
    This tutorial has guided you through implementing the core quantum optimization
    techniques used in the OQI_Project for agricultural optimization. Now it's time
    to step back and consolidate your understanding.
    """
    
    print("🎓 Food Production QAOA Tutorial - Learning Reflection")
    print("=" * 60)
    
    reflection_questions = [
        "How does QUBO formulation translate real-world constraints into quantum-ready problems?",
        "What role does quantum superposition play in exploring farm-food allocation possibilities?",
        "Why is parameter optimization crucial for QAOA success, and what challenges does it face?",
        "How does Recursive QAOA address the scalability limitations of standard QAOA?",
        "Under what conditions do quantum methods outperform classical approaches?",
        "What are the key implementation challenges for practical quantum optimization?"
    ]
    
    print("\n🤔 Key Reflection Questions:")
    for i, question in enumerate(reflection_questions, 1):
        print(f"  {i}. {question}")
    
    practical_applications = [
        "Supply chain optimization with sustainability constraints",
        "Resource allocation in agricultural cooperatives",
        "Multi-objective land use planning",
        "Food security optimization under uncertainty",
        "Crop rotation planning with soil health considerations",
        "Distribution network design for local food systems"
    ]
    
    print("\n🌱 Real-World Applications You Can Now Approach:")
    for app in practical_applications:
        print(f"  • {app}")
    
    implementation_skills = [
        "Converting complex constraints into QUBO penalty terms",
        "Designing quantum circuits for optimization problems",
        "Implementing hybrid classical-quantum optimization loops",
        "Scaling quantum algorithms through recursive decomposition",
        "Benchmarking quantum vs classical performance systematically",
        "Analyzing quantum advantage in practical contexts"
    ]
    
    print("\n💡 Technical Skills Developed:")
    for skill in implementation_skills:
        print(f"  • {skill}")

def implementation_validation_checklist():
    """
    Validate that your implementations cover all essential components.
    
    Use this checklist to ensure your tutorial implementations are complete and correct.
    Your goal is to have working versions of all the key components that could serve
    as the foundation for real applications.
    """
    
    print("\n✅ Implementation Validation Checklist")
    print("=" * 40)
    
    core_components = [
        "FoodProductionQUBO class with complete QUBO formulation methods",
        "FoodProductionQAOA class with quantum state evolution",
        "Parameter optimization with multiple classical methods",
        "Recursive QAOA with confidence measurement and problem reduction",
        "Classical baseline methods for performance comparison",
        "Comprehensive benchmarking and analysis framework",
        "Visualization system for understanding quantum advantage",
        "Integration with real food production data and constraints"
    ]
    
    print("\nCore Components to Verify:")
    for component in core_components:
        print(f"  □ {component}")
    
    testing_scenarios = [
        "Small problems (6-9 variables): Verify correctness against exact solutions",
        "Medium problems (10-16 variables): Compare quantum vs classical performance",
        "Large problems (17+ variables): Test recursive QAOA scalability",
        "Constrained problems: Validate penalty term handling",
        "Multi-objective problems: Verify weighted objective combination",
        "Edge cases: Handle empty solutions, infeasible problems, numerical issues"
    ]
    
    print("\nTesting Scenarios to Complete:")
    for scenario in testing_scenarios:
        print(f"  □ {scenario}")

def next_steps_guidance():
    """
    Provide guidance for continuing your quantum optimization journey.
    
    This tutorial is just the beginning. Here's how to continue building your
    expertise in quantum optimization for real-world applications.
    """
    
    print("\n🚀 Next Steps in Your Quantum Optimization Journey")
    print("=" * 50)
    
    immediate_extensions = [
        "Implement error mitigation techniques for noisy quantum devices",
        "Add more sophisticated constraint handling methods",
        "Extend to dynamic optimization with time-varying constraints",
        "Incorporate uncertainty quantification in the optimization",
        "Develop domain-specific heuristics for agricultural problems",
        "Create user interfaces for non-technical stakeholders"
    ]
    
    advanced_topics = [
        "Quantum-Enhanced Benders Decomposition (next tutorial)",
        "Variational Quantum Eigensolvers for portfolio optimization",
        "Quantum Machine Learning for demand forecasting",
        "Quantum-classical hybrid algorithms for large-scale problems",
        "Hardware-specific optimization for different quantum platforms",
        "Integration with classical optimization software ecosystems"
    ]
    
    research_directions = [
        "Theoretical analysis of QAOA performance on structured problems",
        "Development of problem-specific ansatz circuits",
        "Investigation of quantum advantage boundaries",
        "Hardware-algorithm co-design for optimization applications",
        "Quantum optimization for sustainability and climate applications",
        "Economic analysis of quantum optimization deployment"
    ]
    
    print("\nImmediate Extensions:")
    for ext in immediate_extensions:
        print(f"  • {ext}")
    
    print("\nAdvanced Topics to Explore:")
    for topic in advanced_topics:
        print(f"  • {topic}")
    
    print("\nResearch Opportunities:")
    for direction in research_directions:
        print(f"  • {direction}")

def tutorial_completion_exercise():
    """
    Final exercise to demonstrate mastery of the tutorial concepts.
    
    This capstone exercise challenges you to apply everything you've learned
    to a realistic agricultural optimization scenario.
    """
    
    print("\n🎯 Capstone Exercise: Design Your Own Agricultural Optimization System")
    print("=" * 70)
    
    print("""
Your final challenge: Design a complete optimization system for a specific agricultural scenario.

Choose one of these scenarios and implement a full solution:

1. **Regional Food Security Planning**
   - 10 farms across different climate zones
   - 8 food types with varying nutritional profiles
   - Constraints: water availability, soil suitability, transportation costs
   - Objectives: minimize cost, maximize nutrition, minimize environmental impact

2. **Sustainable Supply Chain Design**
   - 12 production facilities with different capabilities
   - 6 product categories with seasonal demand variation
   - Constraints: capacity limits, quality requirements, sustainability targets
   - Objectives: maximize profit, minimize carbon footprint, ensure supply reliability

3. **Agricultural Cooperative Resource Allocation**
   - 15 member farms with different specializations
   - 5 shared resources (equipment, processing, storage)
   - Constraints: member equity, resource capacity, timing coordination
   - Objectives: maximize collective benefit, ensure fair allocation, minimize conflicts

Your implementation should include:
✓ Complete QUBO formulation with realistic constraints
✓ Both standard and recursive QAOA implementations
✓ Comprehensive comparison with classical methods
✓ Visualization and analysis of results
✓ Practical recommendations for stakeholders
✓ Discussion of implementation challenges and solutions

This capstone project will demonstrate your ability to apply quantum optimization
to real-world agricultural challenges with practical impact!
""")

# Run the reflection and validation exercises
tutorial_reflection_exercise()
implementation_validation_checklist()
next_steps_guidance()
tutorial_completion_exercise()

print("\n🌟 Congratulations on completing the Food Production QAOA Tutorial!")
print("You now have the foundational skills to apply quantum optimization to")
print("agricultural and supply chain challenges using techniques from the OQI_Project.")
print("\nReady for the next challenge? Continue to Quantum-Enhanced Benders Decomposition!")